In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
import spacy
from spacy import displacy
from termcolor import colored
from extraction import findSVOs
import networkx as nx
import matplotlib.pyplot as plt
from topic_modelling import get_topics
import time
from oops import actionTaken, getViolations, getViolators

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [10]:
def identify_crime(text, types, folders):
    text = text.lower()
    data = []
    for i in range(len(types)):
        if(text.find(types[i]) != -1):
            data.append(folders[i])
    return data

In [17]:
df = pd.read_csv('data.csv')

df.Text = df.Text.apply(lambda x : x.replace('\\n', '\n'))
df.Text = df.Text.apply(lambda x : x.replace('\\t', '\t'))

types = ['insider', 'misappropriate', 'unregister', 'fraud']
folders = ['Insider Trading', 'Misappropriated Funds', 'Unregistered', 'Fraud']
crimes = []
actions = []
violations = []
violators = []

for i in range(len(df['Text'])):
    text = df['Text'][i]
    crimes.append(identify_crime(text, types, folders))
    action = actionTaken(text)
    violation = getViolations(text)
    violator = getViolators(text)
    violations.append(violation)
    violators.append(violator)
    actions.append(action)

df['Crimes'] = crimes
df['Violations'] = violations
df['Violators'] = violators
df['Action Taken'] = actions

In [18]:
df.head()

,Text,Crimes,Violations,Violators,Action Taken
0,sec charges at&t and three executives with sel...,[],[violations of the disclosure provisions of Se...,"[at&t, inc., et al.]",[the complaint seeks permanent injunctive reli...
1,u.s. securities and exchange commission\n\nlit...,"[Insider Trading, Fraud]",[violating Section 10(b) of the securities exc...,"[mitchel s. guttenberg, erik r. franklin, davi...",[]
2,u.s. securities and exchange commission\n\nlit...,[Insider Trading],[violating Sections 10(b) and 14(e) of the sec...,"[dr. bobby v. khan, civil cas]",[the commission seeks an order permanently enj...
3,\t\nu.s. securities and exchange commission\n\...,[Insider Trading],[violated Sections 10(b) and 14(e) of the secu...,"[matthew h. kluger and garrett d. bauer, cas]",[]
4,\t\nu.s. securities and exchange commission\n\...,[Insider Trading],[violations of Section 10(b) of the securities...,[],[without admitting or denying the allegations ...


In [19]:
df.Text = df.Text.apply(lambda x : x.replace('\n', '\\n'))
df.Text = df.Text.apply(lambda x : x.replace('\t', '\\t'))

df.to_csv('data.csv', index=False)

In [32]:
for i in range(len(df['Text'])):
    
    final_nodes = []
    
    nodes = []
    nodes.append(df['Violators'][i])
    nodes.append('violated')
    nodes.append(df['Violations'][i])
    final_nodes.append(nodes)
    
    nodes = []
    nodes.append(df['Violators'][i])
    nodes.append('carried out')
    nodes.append(df['Crimes'][i])
    final_nodes.append(nodes)
    
    nodes = []
    nodes.append(df['Violators'][i])
    nodes.append('charged with')
    nodes.append(df['Action Taken'][i])
    final_nodes.append(nodes)
    
    print(final_nodes)
    
    create_graph(final_nodes, 1)

[[['at&t, inc., et al.'], 'violated', ['violations of the disclosure provisions of Section 13(a) of the securities exchange act of 1934 and regulation fd thereunder']], [['at&t, inc., et al.'], 'carried out', []], [['at&t, inc., et al.'], 'charged with', ['the complaint seeks permanent injunctive relief and civil monetary penalties against each defendant.']]]

Total number of extracted pairs: 3

Edges:  ['violated', 'carried out', 'charged with']

Entities:  [[['at&t, inc., et al.'], 'violated', ['violations of the disclosure provisions of Section 13(a) of the securities exchange act of 1934 and regulation fd thereunder']], [['at&t, inc., et al.'], 'carried out', []], [['at&t, inc., et al.'], 'charged with', ['the complaint seeks permanent injunctive relief and civil monetary penalties against each defendant.']]]


TypeError: unhashable type: 'list'

In [31]:
def create_graph(final_nodes, cnt):
    source = []
    target = []
    edge = []
    indexes = []

    for i in (range(len(final_nodes))):
        ent1 = (final_nodes[i][0])
        ent2 = (final_nodes[i][2])
        rel = final_nodes[i][1] 
        source.append(ent1)
        target.append(ent2)
        edge.append("".join(rel).strip())
        indexes.append(i)
    print("\nTotal number of extracted pairs:", len(edge))
    print("\nEdges: ", edge)
    print("\nEntities: ", final_nodes)
    if(len(edge)==0 or len(final_nodes)==0):
        print('False')
    else:
        G = nx.DiGraph(directed=True)
        for i in (range(len(edge))):
            G.add_weighted_edges_from([(source[i],target[i],i)])
        print("\nGraph generated")
        size=20
        if len(edge)/2 > 20:
            size = len(edge)/2
        plt.figure(figsize=(size,size))
        edge_labels=dict([((u,v,),edge[d['weight']]) for u,v,d in G.edges(data=True)])
        pos = nx.spring_layout(G,k=0.8)
        nx.draw(G, with_labels=True, node_color=colors[cnt], node_size=5000, edge_color='r', edge_cmap=plt.cm.Blues, pos=pos, font_size=20)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels, font_size=15)
        return G, edge_labels